In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# 超参数
n_x = 200 # 空间网格数
n_t = 100 # 时间网格数
t_max = 1.0 # 最大时间
x_min, x_max = -10.0, 10.0 # 空间区间
t_min, t_max = 0.0, 1.0 # 时间区间
learning_rate = 0.001 # 学习率
n_hidden = 20 # 隐藏层神经元数
n_epoch = 10000 # 训练轮数

In [14]:
# 生成训练数据
def generate_training_data():
    x = np.linspace(x_min, x_max, n_x)[:, None] # 空间网格
    t = np.linspace(t_min, t_max, n_t)[:, None] # 时间网格
    X, T = np.meshgrid(x, t) # 构建网格
    X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
    u_star = -(X_star[:, 1:2] - 0.5) * np.sin(np.pi * X_star[:, 0:1])
    X_star = torch.from_numpy(X_star).float().requires_grad_(True)
    #X_star = torch.from_numpy(np.vstack([X_star, u_star])).float().requires_grad_(True)

    u_star = torch.from_numpy(u_star).float()
    return X_star, u_star

# 定义网络模型
class PINN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, n_hidden)
        self.fc2 = nn.Linear(n_hidden, n_hidden)
        self.fc3 = nn.Linear(n_hidden, n_hidden)
        self.fc4 = nn.Linear(n_hidden, 1)
        self.tanh = nn.Tanh()

    def forward(self, x):
        a = self.tanh(self.fc1(x))
        a = self.tanh(self.fc2(a))
        a = self.tanh(self.fc3(a))
        u = self.fc4(a)
        return u


In [15]:
# 训练函数
def train_pinn():
    # 生成训练数据
    X_star, u_star = generate_training_data()

    # 初始化 PINN 模型和优化器
    model = PINN()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # 训练循环
    for epoch in range(n_epoch):
        optimizer.zero_grad()
        u_pred = model(X_star)
        u_x, u_t = torch.autograd.grad(u_pred, X_star, grad_outputs=torch.ones_like(u_pred), create_graph=True)
        u_xx = torch.autograd.grad(u_x, X_star, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
        f = u_t + u_pred * u_x - 0.01 * u_xx
        loss = torch.mean(torch.square(u_star - u_pred)) + torch.mean(torch.square(f))
        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print('Epoch: {}, Loss: {:.6f}'.format(epoch, loss.item()))

    # 返回模型和训练数据
    return model, X_star, u_star



In [16]:
# 调用训练函数
model, X_star, u_star = train_pinn()

# 绘制结果
x = np.linspace(x_min, x_max, n_x)
t = np.linspace(t_min, t_max, n_t)
X, T = np.meshgrid(x, t)
U_pred = model(torch.from_numpy(np.hstack((X.flatten()[:, None], T.flatten()[:, None]))).float()).detach().numpy().reshape(n_t, n_x)
U_star = -(T - 0.5) * np.sin(np.pi * X)
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.pcolormesh(X, T, U_pred, cmap='jet')
plt.colorbar()
plt.xlabel('x')
plt.ylabel('t')
plt.title('PINN')
plt.subplot(1, 2, 2)
plt.pcolormesh(X, T, U_star, cmap='jet')
plt.colorbar()
plt.xlabel('x')
plt.ylabel('t')
plt.title('Exact')
plt.show()

ValueError: not enough values to unpack (expected 2, got 1)